In [25]:
path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\office-projects\pdf-extractor"
file_path = r"\data\Tabular_for_extract.pdf";


In [33]:
import camelot
import json
import pandas as pd

In [37]:

def extract_and_save_tables(input_path, csv_path):
    """
    Extract tables from a PDF using camelot, convert to pandas DataFrame, and save as a CSV file.

    Args:
        input_path (str): Path to the input PDF file.
        csv_path (str): Path to the output CSV file.
    """
    # Extract tables using camelot
    tables = camelot.read_pdf(input_path, flavor='stream', pages='all')
    tables.export(csv_path, f = 'csv')
    




In [38]:
extract_and_save_tables(input_path, csv_path)